In [1]:
import heapq

In [3]:
def dijkstra(graph, start):
    # Priority queue to store (distance, node) pairs
    priority_queue = []
    heapq.heappush(priority_queue, (0, start))
    
    # Dictionary to store the shortest distances
    shortest_distances = {node: float('inf') for node in graph}
    shortest_distances[start] = 0
    
    while priority_queue:
        current_distance, current_node = heapq.heappop(priority_queue)

        if current_distance > shortest_distances[current_node]:
            continue

        for neighbor, weight in graph[current_node].items():
            distance = current_distance + weight

            if distance < shortest_distances[neighbor]:
                shortest_distances[neighbor] = distance
                heapq.heappush(priority_queue, (distance, neighbor))

    return shortest_distances

# Example graph representation
graph = {
    'A': {'B': 1, 'C': 4},
    'B': {'A': 1, 'C': 2, 'D': 5},
    'C': {'A': 4, 'B': 2, 'D': 1},
    'D': {'B': 5, 'C': 1}
}

# Run Dijkstra's algorithm from node 'A'
shortest_paths = dijkstra(graph, 'A')
print(shortest_paths)


{'A': 0, 'B': 1, 'C': 3, 'D': 4}


In [4]:
import heapq

def heuristic(node, goal):
    """Example heuristic function using Manhattan distance"""
    return abs(node[0] - goal[0]) + abs(node[1] - goal[1])

def a_star(graph, start, goal):
    priority_queue = []
    heapq.heappush(priority_queue, (0, start))  # (cost, node)
    
    came_from = {}
    cost_so_far = {node: float('inf') for node in graph}
    cost_so_far[start] = 0
    
    while priority_queue:
        _, current = heapq.heappop(priority_queue)
        
        if current == goal:
            break
        
        for neighbor, weight in graph[current].items():
            new_cost = cost_so_far[current] + weight
            if new_cost < cost_so_far[neighbor]:
                cost_so_far[neighbor] = new_cost
                priority = new_cost + heuristic(neighbor, goal)
                heapq.heappush(priority_queue, (priority, neighbor))
                came_from[neighbor] = current
                
    return reconstruct_path(came_from, start, goal)

def reconstruct_path(came_from, start, goal):
    path = []
    current = goal
    while current != start:
        path.append(current)
        current = came_from.get(current, start)
    path.append(start)
    path.reverse()
    return path

# Example graph representation
graph = {
    (0, 0): {(1, 0): 1, (0, 1): 1},
    (1, 0): {(0, 0): 1, (1, 1): 1},
    (0, 1): {(0, 0): 1, (1, 1): 1},
    (1, 1): {(1, 0): 1, (0, 1): 1}
}

# Run A* algorithm
start_node = (0, 0)
goal_node = (1, 1)
path = a_star(graph, start_node, goal_node)
print("Shortest path:", path)


Shortest path: [(0, 0), (0, 1), (1, 1)]


In [ ]:
def bellman_ford(graph, start):
    # Initialize distances from start to all nodes as infinity
    distances = {node: float('inf') for node in graph}
    distances[start] = 0
    
    # Relax all edges |V| - 1 times
    for _ in range(len(graph) - 1):
        for node in graph:
            for neighbor, weight in graph[node].items():
                if distances[node] + weight < distances[neighbor]:
                    distances[neighbor] = distances[node] + weight
                    
    return distances

# Example graph representation
graph = {
    'A': {'B': 1, 'C': 4},
    'B': {'C': 2, 'D': 5},
    'C': {'D': 1},
    'D': {}
}

# Run Bellman-Ford from 'A'
shortest_paths = bellman_ford(graph, 'A')
print("Shortest distances:", shortest_paths)


In [5]:
def floyd_warshall(graph):
    # Initialize distance matrix
    nodes = list(graph.keys())
    num_nodes = len(nodes)
    dist = {node: {neighbor: float('inf') for neighbor in nodes} for node in nodes}
    
    # Set initial distances based on graph edges
    for node in graph:
        dist[node][node] = 0  # Distance to itself is always 0
        for neighbor, weight in graph[node].items():
            dist[node][neighbor] = weight
    
    # Floyd-Warshall algorithm
    for k in nodes:  # Intermediate node
        for i in nodes:  # Start node
            for j in nodes:  # End node
                dist[i][j] = min(dist[i][j], dist[i][k] + dist[k][j])
    
    return dist

# Example graph representation
graph = {
    'A': {'B': 3, 'C': 7},
    'B': {'A': 3, 'C': 1, 'D': 5},
    'C': {'A': 7, 'B': 1, 'D': 2},
    'D': {'B': 5, 'C': 2}
}

# Run Floyd-Warshall
shortest_paths = floyd_warshall(graph)
print("Shortest path matrix:")
for row in shortest_paths:
    print(row, shortest_paths[row])


Shortest path matrix:
A {'A': 0, 'B': 3, 'C': 4, 'D': 6}
B {'A': 3, 'B': 0, 'C': 1, 'D': 3}
C {'A': 4, 'B': 1, 'C': 0, 'D': 2}
D {'A': 6, 'B': 3, 'C': 2, 'D': 0}


In [ ]:
from pulp import LpMaximize, LpProblem, LpVariable, LpMinimize, LpMaximize, LpStatus, lpSum, listSolvers, value, PULP_CBC_CMD, GLPK_CMD

# Define sets of shovels and dumping locations
shovels = ['Shovel1', 'Shovel2', 'Shovel3', 'Shovel4', 'Shovel5', 'Shovel6', 'Shovel7']
crushers = ['Crusher1', 'Crusher2']
stockpiles = ['Stockpile1', 'Stockpile2', 'Stockpile3']
waste_dump = ['WasteDump']
all_dumps = crushers + stockpiles + waste_dump

# Productivity of each shovel (tons per hour)
prod = {'Shovel1': 900, 'Shovel2': 950, 'Shovel3': 720, 'Shovel4': 880, 'Shovel5': 610, 'Shovel6': 1030, 'Shovel7': 460}  # Shovel7 mines waste

# Dumping location capacity (tons per hour)
cap = {'Crusher1': 300, 'Crusher2': 280, 'Stockpile1': 200, 'Stockpile2': 220, 'Stockpile3': 250, 'WasteDump': 500}

# Travel duration cost (minutes per ton)
travel_time    = {
    ('Shovel1', 'Crusher1'): 50, ('Shovel1', 'Crusher2'): 70, ('Shovel1', 'Stockpile1'): 90, ('Shovel1', 'Stockpile2'): 100, ('Shovel1', 'Stockpile3'): 120,
    ('Shovel2', 'Crusher1'): 60, ('Shovel2', 'Crusher2'): 80, ('Shovel2', 'Stockpile1'): 110, ('Shovel2', 'Stockpile2'): 90, ('Shovel2', 'Stockpile3'): 140,
    ('Shovel3', 'Crusher1'): 70, ('Shovel3', 'Crusher2'): 60, ('Shovel3', 'Stockpile1'): 100, ('Shovel3', 'Stockpile2'): 130, ('Shovel3', 'Stockpile3'): 150,
    ('Shovel4', 'Crusher1'): 60, ('Shovel4', 'Crusher2'): 90, ('Shovel4', 'Stockpile1'): 120, ('Shovel4', 'Stockpile2'): 110, ('Shovel4', 'Stockpile3'): 160,
    ('Shovel5', 'Crusher1'): 50, ('Shovel5', 'Crusher2'): 70, ('Shovel5', 'Stockpile1'): 90, ('Shovel5', 'Stockpile2'): 100, ('Shovel5', 'Stockpile3'): 120,
    ('Shovel6', 'Crusher1'): 60, ('Shovel6', 'Crusher2'): 80, ('Shovel6', 'Stockpile1'): 110, ('Shovel6', 'Stockpile2'): 90, ('Shovel6', 'Stockpile3'): 140,
    ('Shovel7', 'WasteDump'): 40  # Waste shovel only goes to WasteDump
}

# Shift duration (hours)
T_shift = 10

# Total hauled tonnage for the shift
total_tonnage = 75000

# Define the optimization problem
model = LpProblem("Mining_Optimization", LpMinimize)

# Define decision variables
x = { (i, j): LpVariable(f"x_{i}_{j}", lowBound=0) for i in shovels for j in all_dumps if (i, j) in travel_time }


# Objective function: Minimize total travel duration
#model += lpSum(travel_time[i, j] * x[i, j] for i in shovels for j in all_dumps if (i, j) in travel_time)
#model += lpSum(travel_time[i, j] * X[i,j] for i in shovels for j in all_dumps if (i, j) in travel_time)
model += lpSum(travel_time[i, j] * x[i, j] for i in shovels for j in all_dumps if (i, j) in travel_time)



# Constraints
# Shovel production limits
for i in shovels:
    model += lpSum(x[i, j] for j in all_dumps if (i, j) in travel_time) <= prod[i], f"ShovelLimit_{i}"

# Dumping location capacity constraints
for j in all_dumps:
    model += lpSum(x[i, j] for i in shovels if (i, j) in travel_time) <= cap[j], f"DumpCapacity_{j}"

# Total hauled tonnage constraint
model += lpSum(x[i, j] for i in shovels for j in all_dumps if (i, j) in travel_time) * T_shift == total_tonnage, "TotalTonnageConstraint"

# Waste material constraint: Any material from Shovel7 must only go to WasteDump
model += lpSum(x['Shovel7', j] for j in all_dumps if (('Shovel7', j) in travel_time and j != 'WasteDump')) == 0, "WasteShovelConstraint"

# Crusher prioritization: Ensure crushers receive more tonnage before stockpiles
for j in stockpiles:
    model += lpSum(x[i, j] for i in shovels if (i, j) in travel_time) <= lpSum(x[i, c] for i in shovels for c in crushers if (i, c) in travel_time), f"CrusherPriority_{j}"




# Additional constraints to prevent negative flows
# for i in shovels:
#     for j in all_dumps:
#         if (i, j) in travel_time:
#             model += x[i, j] >= 0, f"NonNegativeFlow_{i}_{j}"

#Instead of this model += lpSum(x[i, j] for i in shovels for j in all_dumps if (i, j) in travel_time) * T_shift == total_tonnage
# Do this - model += lpSum(x[i, j] for i in shovels for j in all_dumps if (i, j) in travel_time) * T_shift <= total_tonnage

#model.solve(PULP_CBC_CMD(msg=True, options=['presolve', 'strong', 'numerical']))
model.solve()

# Solve the model
#model.solve()

# Print results
for i in shovels:
    for j in all_dumps:
        if (i, j) in travel_time:
            print(f"Material transported from {i} to {j}: {x[i, j].varValue} tons per hour")

print(f"Solution Status: {LpStatus[prob.status]}")
print(f"Total transportation cost: {model.objective.value()}")

# **Step 2: Check Redundancy by Removing Constraints One at a Time**
redundant_constraints = []
original_constraints = list(model.constraints.keys())  # Store constraint names

for constraint_name in original_constraints:
    removed_constraint = model.constraints.pop(constraint_name)  # Remove by name
    model.solve(PULP_CBC_CMD())  # Re-solve after removal
    new_objective = value(model.objective)

    if new_objective == full_objective:  # If objective remains unchanged, constraint is redundant
        redundant_constraints.append(constraint_name)

    # Restore constraint
    model.constraints[constraint_name] = removed_constraint

# **Step 3: Report Redundant Constraints**

if redundant_constraints:
    print("\nRedundant Constraints Found:")
    for rc in redundant_constraints:
        print(f"- {rc}")
else:
    print("No redundant constraints detected.")



In [ ]:
# Define sets of shovels and dumping locations
shovels = ['Shovel1', 'Shovel2']
dumps = ['Dump1', 'Dump2']

# Productivity of each shovel (tons per hour)
prod = {'Shovel1': 200, 'Shovel2': 250}

# Dumping location capacity (tons per hour)
cap = {'Dump1': 180, 'Dump2': 270}

# Travel duration cost (minutes per ton)
travel_time = {
    ('Shovel1', 'Dump1'): 25,
    ('Shovel1', 'Dump2'): 28,
    ('Shovel2', 'Dump1'): 27,
    ('Shovel2', 'Dump2'): 26
}

# Shift duration (hours)
T_shift = 8

# Total hauled tonnage for the shift
total_tonnage = 3000

# Define the optimization problem
model = LpProblem("Mining_Optimization", LpMinimize)

# Define decision variables
x = { (i, j): LpVariable(f"x_{i}_{j}", lowBound=0) for i in shovels for j in dumps }

# Objective function: Minimize total travel duration
model += lpSum(travel_time[i, j] * x[i, j] for i in shovels for j in dumps)

# Constraints
# Shovel production limits
for i in shovels:
    model += lpSum(x[i, j] for j in dumps) <= prod[i], f"ShovelLimit_{i}"

# Dumping location capacity constraints
for j in dumps:
    model += lpSum(x[i, j] for i in shovels) <= cap[j], f"DumpCapacity_{j}"

# Total hauled tonnage constraint
model += lpSum(x[i, j] for i in shovels for j in dumps) * T_shift == total_tonnage, "TotalTonnageConstraint"

# Solve the model
model.solve()

# Print results
for i in shovels:
    for j in dumps:
        print(f"Material transported from {i} to {j}: {x[i, j].varValue} tons per hour")

print(f"Solution Status: {LpStatus[prob.status]}")
print(f"Total transportation cost: {model.objective.value()}")


In [ ]:
from pulp import LpMinimize, LpProblem, LpVariable, lpSum, LpStatus, value, PULP_CBC_CMD

# Define sets of shovels and dumping locations
shovels = ['Shovel1', 'Shovel2', 'Shovel3', 'Shovel4', 'Shovel5', 'Shovel6', 'Shovel7']
crushers = ['Crusher1', 'Crusher2']
stockpiles = ['Stockpile1', 'Stockpile2', 'Stockpile3']
waste_dump = ['WasteDump']
all_dumps = crushers + stockpiles + waste_dump

# Productivity of each shovel (tons per hour)
prod = {'Shovel1': 12000, 'Shovel2': 250, 'Shovel3': 220, 'Shovel4': 180, 'Shovel5': 210, 'Shovel6': 230, 'Shovel7': 260}

# Dumping location capacity (tons per hour)
cap = {'Crusher1': 300, 'Crusher2': 280, 'Stockpile1': 200, 'Stockpile2': 220, 'Stockpile3': 250, 'WasteDump': 500}

# Travel duration cost (minutes per ton)
travel_time = {
    ('Shovel1', 'Crusher1'): 15, ('Shovel1', 'Crusher2'): 18, ('Shovel1', 'Stockpile1'): 25,
    ('Shovel2', 'Crusher1'): 17, ('Shovel2', 'Crusher2'): 20, ('Shovel2', 'Stockpile1'): 27,
    ('Shovel3', 'Crusher1'): 19, ('Shovel3', 'Crusher2'): 16, ('Shovel3', 'Stockpile1'): 26,
    ('Shovel4', 'Crusher1'): 18, ('Shovel4', 'Crusher2'): 22, ('Shovel4', 'Stockpile1'): 28,
    ('Shovel5', 'Crusher1'): 15, ('Shovel5', 'Crusher2'): 18, ('Shovel5', 'Stockpile1'): 25,
    ('Shovel6', 'Crusher1'): 17, ('Shovel6', 'Crusher2'): 20, ('Shovel6', 'Stockpile1'): 27,
    ('Shovel7', 'WasteDump'): 12  # Waste shovel only goes to WasteDump
}

# Shift duration (hours)
T_shift = 10

# Total hauled tonnage for the shift
total_tonnage = 60000

# Define the optimization problem
model = LpProblem("Mining_Optimization", LpMinimize)

# Define decision variables
x = {(i, j): LpVariable(f"x_{i}_{j}", lowBound=0) for i in shovels for j in all_dumps if (i, j) in travel_time}

# Objective function: Minimize total travel duration
model += lpSum(travel_time[i, j] * x[i, j] for i in shovels for j in all_dumps if (i, j) in travel_time)

# Constraints (Named for easier removal & restoration)
for i in shovels:
    model += lpSum(x[i, j] for j in all_dumps if (i, j) in travel_time) <= prod[i], f"ShovelLimit_{i}"

for j in all_dumps:
    model += lpSum(x[i, j] for i in shovels if (i, j) in travel_time) <= cap[j], f"DumpCapacity_{j}"

model += lpSum(x[i, j] for i in shovels for j in all_dumps if (i, j) in travel_time) * T_shift <= total_tonnage, "TotalTonnageConstraint"

model += lpSum(x['Shovel7', j] for j in all_dumps if (('Shovel7', j) in travel_time and j != 'WasteDump')) == 0, "WasteShovelConstraint"

# Solve the full problem
model.solve(PULP_CBC_CMD())
full_objective = value(model.objective)

print(f"Original Objective Value: {full_objective}")
print(f"Solution Status: {LpStatus[model.status]}")

# **Step 2: Check Redundancy by Removing Constraints One at a Time**
redundant_constraints = []
original_constraints = list(model.constraints.keys())  # Store constraint names

for constraint_name in original_constraints:
    removed_constraint = model.constraints.pop(constraint_name)  # Remove by name
    model.solve(PULP_CBC_CMD())  # Re-solve after removal
    new_objective = value(model.objective)

    if new_objective == full_objective:  # If objective remains unchanged, constraint is redundant
        redundant_constraints.append(constraint_name)

    # Restore constraint
    model.constraints[constraint_name] = removed_constraint

# **Step 3: Report Redundant Constraints**
print("\nRedundant Constraints Found:")
if redundant_constraints:
    for rc in redundant_constraints:
        print(f"- {rc}")
else:
    print("No redundant constraints detected.")


In [ ]:
from pulp import LpMinimize, LpProblem, LpVariable, lpSum, PULP_CBC_CMD

# Define sets of shovels and dumping locations
shovels = ['Shovel1', 'Shovel2', 'Shovel3', 'Shovel4', 'Shovel5', 'Shovel6', 'Shovel7']
crushers = ['Crusher1', 'Crusher2']
stockpiles = ['Stockpile1', 'Stockpile2', 'Stockpile3']
waste_dump = ['WasteDump']
all_dumps = crushers + stockpiles + waste_dump

# Productivity of each shovel (tons per hour)
prod = {'Shovel1': 12000, 'Shovel2': 250, 'Shovel3': 220, 'Shovel4': 180, 'Shovel5': 210, 'Shovel6': 230, 'Shovel7': 260}  # Shovel7 mines waste

# Dumping location capacity (tons per hour)
cap = {'Crusher1': 300, 'Crusher2': 280, 'Stockpile1': 200, 'Stockpile2': 220, 'Stockpile3': 250, 'WasteDump': 500}

# Travel duration cost (total minutes)
travel_time = {
    ('Shovel1', 'Crusher1'): 15, ('Shovel1', 'Crusher2'): 18, ('Shovel1', 'Stockpile1'): 25, ('Shovel1', 'Stockpile2'): 28, ('Shovel1', 'Stockpile3'): 30,
    ('Shovel2', 'Crusher1'): 17, ('Shovel2', 'Crusher2'): 20, ('Shovel2', 'Stockpile1'): 27, ('Shovel2', 'Stockpile2'): 29, ('Shovel2', 'Stockpile3'): 32,
    ('Shovel3', 'Crusher1'): 19, ('Shovel3', 'Crusher2'): 16, ('Shovel3', 'Stockpile1'): 26, ('Shovel3', 'Stockpile2'): 30, ('Shovel3', 'Stockpile3'): 34,
    ('Shovel4', 'Crusher1'): 18, ('Shovel4', 'Crusher2'): 22, ('Shovel4', 'Stockpile1'): 28, ('Shovel4', 'Stockpile2'): 30, ('Shovel4', 'Stockpile3'): 36,
    ('Shovel5', 'Crusher1'): 15, ('Shovel5', 'Crusher2'): 18, ('Shovel5', 'Stockpile1'): 25, ('Shovel5', 'Stockpile2'): 27, ('Shovel5', 'Stockpile3'): 30,
    ('Shovel6', 'Crusher1'): 17, ('Shovel6', 'Crusher2'): 20, ('Shovel6', 'Stockpile1'): 27, ('Shovel6', 'Stockpile2'): 29, ('Shovel6', 'Stockpile3'): 32,
    ('Shovel7', 'WasteDump'): 12  # Waste shovel only goes to WasteDump
}

# Shift duration (hours)
T_shift = 10

# Total hauled tonnage for the shift
total_tonnage = 60000  # Large-scale test case - each crusher at 2000 tons/hour for 10 hours shift = 40000 tons. Remainder goes to stock piles (2 iron ore)

# Define the optimization problem
model = LpProblem("Mining_Optimization", LpMinimize)

# Define decision variables
x = { (i, j): LpVariable(f"x_{i}_{j}", lowBound=0) for i in shovels for j in all_dumps if (i, j) in travel_time }

# Objective function: Minimize total travel duration + crusher balance penalty
balance_diff = LpVariable("BalanceDiff", lowBound=0)
model += lpSum(travel_time[i, j] * x[i,j] for i in shovels for j in all_dumps if (i, j) in travel_time) + balance_diff

# Constraints
# Shovel production limits
for i in shovels:
    model += lpSum(x[i, j] for j in all_dumps if (i, j) in travel_time) <= prod[i], f"ShovelLimit_{i}"

# Dumping location capacity constraints
for j in all_dumps:
    model += lpSum(x[i, j] for i in shovels if (i, j) in travel_time) <= cap[j], f"DumpCapacity_{j}"

# Total hauled tonnage constraint
model += lpSum(x[i, j] for i in shovels for j in all_dumps if (i, j) in travel_time) * T_shift <= total_tonnage, "TotalTonnageConstraint"

# Waste material constraint: Any material from Shovel7 must only go to WasteDump
model += lpSum(x['Shovel7', j] for j in all_dumps if (('Shovel7', j) in travel_time and j != 'WasteDump')) == 0, "WasteShovelConstraint"

# Crusher prioritization: Ensure crushers receive more ore before stockpiles
for j in stockpiles:
    model += lpSum(x[i, j] for i in shovels if (i, j) in travel_time) <= 0.1 * (cap["Crusher1"] + cap["Crusher2"]), f"CrusherPriority_{j}"

model += balance_diff >= lpSum(x[i, "Crusher1"] for i in shovels if (i, "Crusher1") in x) - lpSum(x[i, "Crusher2"] for i in shovels if (i, "Crusher2") in x)


# Crushers must be balanced as evenly as possible
# model += balance_diff >= lpSum(x[i, "Crusher1"] for i in shovels) - lpSum(x[i, "Crusher2"] for i in shovels)
# model += balance_diff >= lpSum(x.get((i, "Crusher1"), 0) for i in shovels) - lpSum(x.get((i, "Crusher2"), 0) for i in shovels)

#model += balance_diff >= lpSum(x[i, "Crusher2"] for i in shovels) - lpSum(x[i, "Crusher1"] for i in shovels)

# Solve the model with PuLP CBC solver for better precision
model.solve(PULP_CBC_CMD(msg=True, options=['presolve', 'strong', 'numerical']))

# Print results
for i in shovels:
    for j in all_dumps:
        if (i, j) in travel_time:
            print(f"Material transported from {i} to {j}: {x[i, j].varValue} tons per hour")
print(f"Solution Status: {LpStatus[prob.status]}")
print(f"Total transportation cost: {model.objective.value()}")

# Test for redundant constraints

